## 仅供交叉验证 LightGBM

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from utils import *
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import StratifiedKFold

np.random.seed(42)
n_input=11
random.seed(42)

读取数据

In [6]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [7]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


## 规范数据并获取5折交叉检验所需的训练集和验证集

In [8]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 构建模型，训练并评估

In [17]:
def cross_validation(X, y, y_cat, kfold, scaler):
    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    tests = []
    preds = []

    for train, test in kfold.split(X, y_cat):
        model = MultiOutputRegressor(lgb.LGBMRegressor(n_estimators=500))
        model.fit(X[train].reshape(len(train), -1), y[train])

        y_test = y[test]
        y_pred = model.predict(X[test].reshape(len(test), -1)).reshape(y[test].shape)

        tests.append(y_test)
        preds.append(y_pred)

        metrics = ['mae', 'rmse','ndcg', 'mape', 'r2', 'pearson', 'acc']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
            
    return overall_metrics, annual_metrics, tests, preds

In [18]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        overall_metrics, annual_metrics, tests, preds = cross_validation(X, y, y_cat, kfold, scaler)
        pickle.dump(tests, open('tree_tests_{}.list'.format(name), 'wb'))
        pickle.dump(preds, open('tree_preds_{}.list'.format(name), 'wb'))
        
        for n in [1, 5, 10, 15, 20, 25, 30, 35, 40]:
            overall_total = []

            for test, pred in zip(tests, preds):
                overall, _ = eval_model('ndcg', test, pred, scaler, n)
                overall_total.append(overall)
            print(n, np.mean(overall_total))
        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
        
        print('=====')
    
    pickle.dump(metrics, open('tree_metrics.dict', 'wb'))
    
    return metrics

In [19]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)
1 0.470121316089885
5 0.5896512543805762
10 0.5898456570429753
15 0.6096437513771067
20 0.6096930177727906
25 0.6102133071770488
30 0.6102128410329387
35 0.6359651574007329
40 0.6361579516377074
=====
1 0.4474773277820588
5 0.4556405643982666
10 0.5085879438114974
15 0.5175935675790458
20 0.5630817064248428
25 0.5692689985490433
30 0.5737181989187838
35 0.5741845740637788
40 0.5749441984497916
=====


In [20]:
metrics

{'gene': {'overall': {'mae': 0.8209942683708282,
   'rmse': 1.3337560891982014,
   'ndcg': 0.6096930177727906,
   'mape': 6.2061780731245095,
   'r2': 0.18674387348607444,
   'pearson': 0.4866112033720305,
   'acc': 0.34014186441579003},
  'annual': {'mae': array([0.45741282, 0.62247792, 0.84728625, 1.00330621, 1.17448814]),
   'rmse': array([0.78222512, 0.99470613, 1.38349926, 1.52179826, 1.73490588]),
   'ndcg': array([0.66153224, 0.32535838, 0.27309157, 0.33757616, 0.10942672]),
   'mape': array([4.68375892, 5.14610333, 5.18272873, 8.72367751, 7.29462188]),
   'r2': array([ 0.37751421,  0.23287471,  0.1018908 ,  0.00455022, -0.06371026]),
   'pearson': array([0.6642235 , 0.55230922, 0.42111769, 0.31195175, 0.22437103]),
   'acc': array([0.59213718, 0.33974633, 0.34467135, 0.23189974, 0.19225472])}},
 'transplant': {'overall': {'mae': 0.820901655306564,
   'rmse': 1.3020615560087556,
   'ndcg': 0.5630817064248428,
   'mape': 3.418630385729486,
   'r2': 0.3946391232084295,
   'pearson

In [8]:
metrics

{'gene': {'overall': {'mae': 0.8209942683708282,
   'rmse': 1.3337560891982014,
   'ndcg': 0.6096930177727906},
  'annual': {'mae': array([0.45741282, 0.62247792, 0.84728625, 1.00330621, 1.17448814]),
   'rmse': array([0.78222512, 0.99470613, 1.38349926, 1.52179826, 1.73490588]),
   'ndcg': array([0.66153224, 0.32535838, 0.27309157, 0.33757616, 0.10942672])}},
 'transplant': {'overall': {'mae': 0.820901655306564,
   'rmse': 1.3020615560087556,
   'ndcg': 0.5630817064248428},
  'annual': {'mae': array([0.80399247, 0.82789492, 0.79520697, 0.80899437, 0.86841955]),
   'rmse': array([1.28646968, 1.34488998, 1.26323874, 1.24789521, 1.3557784 ]),
   'ndcg': array([0.20565586, 0.02751808, 0.01730095, 0.09461803, 0.12595979])}}}

In [5]:
metrics

{'gene': {'overall': {'mae': 0.8209942683708282,
   'rmse': 1.3337560891982014,
   'ndcg': 0.6096930177727906,
   'mape': 6.2061780731245095,
   'r2': 0.18674387348607444,
   'pearson': 0.4866112033720305,
   'acc': 0.34014186441579003},
  'annual': {'mae': array([0.45741282, 0.62247792, 0.84728625, 1.00330621, 1.17448814]),
   'rmse': array([0.78222512, 0.99470613, 1.38349926, 1.52179826, 1.73490588]),
   'ndcg': array([0.66153224, 0.32535838, 0.27309157, 0.33757616, 0.10942672]),
   'mape': array([4.68375892, 5.14610333, 5.18272873, 8.72367751, 7.29462188]),
   'r2': array([ 0.37751421,  0.23287471,  0.1018908 ,  0.00455022, -0.06371026]),
   'pearson': array([0.6642235 , 0.55230922, 0.42111769, 0.31195175, 0.22437103]),
   'acc': array([0.59213718, 0.33974633, 0.34467135, 0.23189974, 0.19225472])}},
 'transplant': {'overall': {'mae': 0.820901655306564,
   'rmse': 1.3020615560087556,
   'ndcg': 0.5630817064248428,
   'mape': 3.418630385729486,
   'r2': 0.3946391232084295,
   'pearson